I have a dataframe (df). It contains predicted daily data from a model, up until the end of 2020. As each day passes in the year, actual and id data is added to the row. There are multiple names for each day

I want to add an additional column named payout. The payout should be 0 unless the sum of actual, month to date has passed the sum of predicted.

I.e., for Nir, we can see the sum of predicted is 4200. So the payout should be 0 until the sum of actual passes 4200. Once that threshold is passed, then the payout should be 1% of actual-predicted. With the above data, 

In [16]:
import pandas as pd

df = pd.read_excel(r'C:\Users\u300286\Desktop\1.xlsx')
df

,NAME,ID,PREDICTED,ACTUAL,YYYY_MM_DD
0,Nir,215,100,400,2020-01-01
1,Nir,215,200,400,2020-01-02
2,Nir,215,100,400,2020-01-03
3,Nir,215,200,400,2020-01-04
4,Nir,215,100,400,2020-01-05
5,Nir,215,200,400,2020-01-06
6,Nir,215,100,400,2020-01-07
7,Nir,215,200,400,2020-01-08
8,Nir,215,100,400,2020-01-09
9,Nir,215,200,400,2020-01-10


In [21]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'Name':['Nir','Nir','Nir','Nir','Xyc','Xyc','Xyc'],'PREDICTED':[100,200,100,200,100,200,300],
                   'ACTUAL':[400,400,400,400,500,500,500],
                   'YYYY_MM_DD':['2020-01-01','2020-01-02','2020-01-03','2020-01-04','2020-01-01','2020-01-02','2020-01-03']})


def calculate(item):
    # select name
    data = df[df['Name'] == item]
    # calculate sum
    sum = data['PREDICTED'].sum()

    # remove NaN rows
    data = data.dropna()

    # calculate and insert  month to date column values
    month_to_date = []
    value = 0
    for index, row in data.iterrows():
        value += row['ACTUAL']
        month_to_date.append(value)

    data.insert(3, "MONTH_TO_DATE", month_to_date, True)

    # calculate and instert payout values
    conditions = [
        (data['MONTH_TO_DATE'] < sum),
        (data['MONTH_TO_DATE'] >= sum)
    ]
    choices = [0, ((data['ACTUAL'] - data['PREDICTED'])/100).astype(int)]
    data.insert(5, "PAYOUT", np.select(conditions, choices), True)

    return data


# collect results
results = pd.DataFrame(columns=['Name','PREDICTED','ACTUAL','MONTH_TO_DATE','YYYY_MM_DD','PAYOUT'])

for item in df['Name'].unique():
    df2 = calculate(item)
    results = results.append(df2)
    
results

,Name,PREDICTED,ACTUAL,MONTH_TO_DATE,YYYY_MM_DD,PAYOUT
0,Nir,100,400,400,2020-01-01,0
1,Nir,200,400,800,2020-01-02,2
2,Nir,100,400,1200,2020-01-03,3
3,Nir,200,400,1600,2020-01-04,2
4,Xyc,100,500,500,2020-01-01,0
5,Xyc,200,500,1000,2020-01-02,3
6,Xyc,300,500,1500,2020-01-03,2
